In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
def plotCorrelation(data):
    corr_data = data.copy()
    names = list(corr_data.columns)
    correlations = corr_data.corr().abs()
    fig = plt.figure(figsize=(50, 50))
    ax = fig.add_subplot(111)
    cax = ax.matshow(correlations, vmin=-1, vmax=1)
    # fig.colorbar(cax)
    ticks = np.arange(0,len(names),1)
    ax.set_xticks(ticks)
    ax.set_yticks(ticks)
    ax.set_xticklabels(names)
    ax.set_yticklabels(names)
    plt.show()
    
def reduce(data, threshold=0.9):
    correlations = data.corr().abs()
    upper = correlations.where(np.triu(np.ones(correlations.shape), k=1).astype(np.bool))
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    return data.drop(columns=to_drop)

def inpute(data, feature):
    from sklearn.linear_model import ElasticNet
    from sklearn.neural_network import MLPRegressor
    
    X = data.copy().drop(columns=[feature])
    X = X.select_dtypes(exclude=['object'])
    X = X.fillna(X.median())
    y = data[feature]
    X_train = X[~y.isna()]
    X_test = X[y.isna()]
    y_train = y[~y.isna()]
    y_test = y[y.isna()]
    
    model = MLPRegressor()
    model.fit(X_train, y_train)
    print("Feature: %s | Loss = " % feature, model.loss_)
    filled_gaps = model.predict(X_test)
    for i, index in enumerate(data[feature][data[feature].isna()].index):
        data.at[index, feature] = filled_gaps[i]
    return data

In [3]:
data = pd.read_csv("data/week-one/X_train.csv")

In [4]:
data['year'] = pd.DatetimeIndex(pd.to_datetime(data.timestamp)).year
data.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc,year
0,11180,2013-10-11,51,30.0,2.0,17.0,1.0,1986.0,2.0,8.0,...,1,0,7,33,1,9,48,11,9300000,2013
1,23071,2014-09-30,48,48.0,9.0,17.0,4.0,2016.0,1.0,10.0,...,5,0,25,44,0,12,114,13,5277083,2014
2,18942,2014-05-27,77,46.0,2.0,16.0,1.0,1978.0,3.0,10.0,...,0,0,5,22,0,1,33,8,12500000,2014
3,24805,2014-11-17,109,54.0,9.0,14.0,1.0,2001.0,3.0,11.0,...,1,0,1,10,1,0,16,0,13050000,2014
4,14778,2014-02-10,33,19.0,2.0,3.0,2.0,1961.0,1.0,6.0,...,3,0,14,29,1,4,90,6,5500000,2014


In [5]:
# for column in data.columns:
#     print(column)

In [6]:
# data.corr()

In [7]:
# plotCorrelation(data)

In [8]:
# data.corr()['price_doc'].sort_values(ascending=False).iloc[:20].plot.bar()


In [9]:
data = reduce(data.drop(columns=['price_doc', 'id']), 0.9)
# data

In [10]:
data.shape

(21329, 145)

In [11]:
# plotCorrelation(data)

In [12]:
# for i in data.dtypes:
#     print(i)

object
int64
float64
float64
float64
float64
float64
float64
float64
float64
object
object
float64
int64
float64
float64
float64
int64
int64
float64
int64
int64
int64
int64
object
int64
int64
int64
object
object
object
object
object
object
object
object
int64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
int64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
int64
float64
float64
object
float64
float64
float64
int64
object
float64
int64
float64
object
int64
float64
int64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
object
float64
float64
int64
int64
int64
int64
int64
float64
int64
int64
int64
int64
int64
int64
int64
float64
float64
int64
int64
int64
float64
int64
int64
int64
int64
float64
int64
int64
float64
int64
int64
int64
int64
float64
int64
int64
float64
int64
float64
int64
int64
float64


In [13]:
for column in data.columns[data.isna().any() == True]:
    data = inpute(data, column)

Loss =  856984.417150649
Loss =  1297328.5401327359
Loss =  36285050.67393486
Loss =  323820.3129652989
Loss =  15330178.156866785
Loss =  18226238.534204032
Loss =  105120635.28484547
Loss =  633662.1365060451
Loss =  9230711.48681801
Loss =  6852582.100154488
Loss =  1718606.8736412805
Loss =  929439.1239866063
Loss =  3165569.124033597
Loss =  3830519.5585088874
Loss =  1222443.6028321746
Loss =  1523055.3690811307
Loss =  5084090.077297843
Loss =  971709.8670524382
Loss =  36657001.24439356
Loss =  1291154.0959216603
Loss =  2930459.8431763425
Loss =  383240.2881778106
Loss =  423153.63022363215
Loss =  629135745.9210757
Loss =  193665.15766596244
Loss =  2721713.5729013933
Loss =  1083770.3377930592
Loss =  593560.516504453
Loss =  30430170.977107655
Loss =  21710487.735671125
Loss =  435501.72492827894
Loss =  536142298.67647195
